In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import lightgbm as lgb
from matplotlib import pyplot as plt
from matplotlib import rcParams
import numpy as np
import pandas as pd
from pathlib import Path
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler, RobustScaler # 데이터 표준화

import seaborn as sns
import warnings
import math
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
# Decision Tree model
from sklearn.tree import DecisionTreeClassifier

# lightgbm 모델
from lightgbm import LGBMClassifier, plot_importance

# Xgboost 모델
import xgboost as xgb

# 랜덤포레스트 
from sklearn.ensemble import RandomForestClassifier

# 로지스틱 리그레션
from sklearn.linear_model import LogisticRegression

# normalization
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

#feature Selection
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import f_classif
import itertools

from sklearn.decomposition import PCA
from sklearn.decomposition import IncrementalPCA
from sklearn.decomposition import KernelPCA

# from google.colab import files

In [5]:
# from tensorflow.keras import models
# from tensorflow.keras import layers
# import tensorflow as tf
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.layers import *
from sklearn.preprocessing import *
# from tensorflow.keras.utils import to_categorical
# from tensorflow.keras.callbacks import EarlyStopping

In [6]:
rcParams['figure.figsize'] = (18, 9)
plt.style.use('fivethirtyeight')
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
warnings.simplefilter('ignore')

In [7]:
data_dir = Path('../input/')
trn_file = data_dir/'fg_trn.csv'
tst_file = data_dir/'fg_tst.csv'
# feature_file = data_dir / 'feature.csv'
sample_file = data_dir/'sample_submission.csv'

submission_dir = Path('../submission')

st_data_dir = Path('../st_input')

target_col = 'class'
seed = 42

In [8]:
def Standardzation_df(df):
    X = RobustScaler().fit_transform(df)
    return pd.DataFrame(X)

In [9]:
def Normalization_df(df):
    X = MinMaxScaler().fit_transform(df)
    return pd.DataFrame(X)

## 앙상블 전 데이터 불러오기

In [10]:
trn = pd.read_csv(trn_file, index_col=0)
print(trn.shape)
trn.head()

(318740, 91)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,class
0,23.2640,20.3368,19.0095,17.6724,16.9396,-8.1086e-05,17.6321,2.9272,23.2640,3.0062,1.3273,2.6643,-2.7875,1.3817,1.3371,2.0699,19.0096,-4.1148,-1.2483,1.3774,17.6725,-2.5854,-1.2827,-2.0155,-0.6925,0.0307,-17.6322,-16.9090,2.8665,4.1691,1.3027,2.6257,1.3230,2.0462,0.7232,1.1439,-2.8690e+05,1.0698,-2.5080e+05,0.8795,1.0729,1.0757,0.8221,0.9384,1.0433,-2.1795e+05,0.8724,0.9323,-2.0891e+05,0.8937,1.0018,-3.5065e-06,-4.0027e-06,-4.2778e-06,-4.5988e-06,-4.7955e-06,-4.5048e-06,-4.5048e-06,-6.8153e-05,-6.8101e-05,-6.8177e-05,-6.8128e-05,1.1415,1.2199,1.0687,1.1489,1.0750,1.1210,1.0428,4.1012e+05,-44588.1645,700.0365,-705.3298,-2043.2987,6.3963,-1.8975,0.1039,-1.1943,-0.1168,-0.3289,-1.2392,0.8164,23.2640,6.3243,6.2153,2.4562,0.0017,0.0307,91.2733,21.9336,0.0
1,15.0521,14.0620,13.4524,13.2684,13.1689,4.5061e-03,13.2363,0.9902,15.0476,1.0476,0.6096,0.7935,-0.9045,0.6506,0.1840,0.2835,13.4479,-1.5140,-0.5521,0.2162,13.2639,-0.7361,-0.1429,-0.2424,-0.0673,0.0343,-13.2317,-13.1302,0.9619,1.5550,0.5931,0.7683,0.1751,0.2767,0.1016,1.0704,3.3404e+03,1.0453,3.1207e+03,0.9396,1.0485,1.0139,0.8988,0.9606,1.0076,2.9446e+03,0.9474,0.9893,2.9225e+03,0.9819,1.0026,3.0108e-04,3.2176e-04,3.3599e-04,3.4043e-04,3.4307e-04,4.5061e-03,4.5061e-03,3.5955e-03,3.5824e-03,3.6020e-03,3.5890e-03,1.0687,1.1159,1.0442,1.0580,1.0132,1.0211,1.0077,-1.3453e+05,1329.3993,234.1073,-139.3329,141.9778,6.1358,0.9277,-2.3727,0.1881,-0.6354,-0.4376,1.1678,-0.0957,15.0521,1.8832,1.8317,0.7586,0.0091,0.0343,62.7375,13.7086,1.0
2,16.7864,15.8254,15.5363,15.3935,15.3500,4.7198e-04,15.3217,0.9610,16.7859,1.0998,0.2891,0.4319,-0.7822,0.3854,0.1428,0.1863,15.5358,-1.0713,-0.1503,0.2146,15.3931,-0.2931,-0.0465,-0.0900,0.0283,0.0540,-15.3213,-15.2956,0.9211,1.1676,0.2466,0.3649,0.1183,0.1440,0.0257,1.0607,3.5566e+04,1.0186,3.3530e+04,0.9529,1.0250,1.0093,0.9355,0.9904,1.0028,3.2615e+04,0.9813,0.9970,3.2523e+04,0.9942,1.0035,2.8419e-05,3.0088e-05,3.0568e-05,3.0804e-05,3.0856e-05,2.3599e-04,2.3599e-04,4.6159e-04,4.6087e-04,4.6194e-04,4.6123e-04,1.0587,1.0756,1.0160,1.0238,1.0077,1.0094,1.0017,-2.0247e+05,1399.9650,333.2776,117.0187,206.6743,8.5752,0.3751,-0.9788,0.0770,-0.7371,0.0022,0.2375,0.3560,16.7864,1.4364,1.3116,0.5462,0.0031,0.0540,73.7791,15.5835,0.0
3,25.6606,21.1887,20.2212,19.8949,19.6346,5.8143e-06,19.7947,4.4719,25.6606,4.6660,0.9676,1.2938,-4.1649,1.1015,0.3262,0.5865,20.2212,-5.1325,-0.7735,0.4265,19.8949,-1.0997,-0.1923,-0.4526,-0.1600,0.0795,-19.7946,-19.5552,4.3590,5.2664,0.9074,1.2000,0.2926,0.5321,0.2395,1.2111,4.4134e+06,1.0478,3.6442e+06,0.8357,1.0548,1.0164,0.7976,0.9632,1.0133,3.4217e+06,0.9476,0.9904,3.3770e+06,0.9775,1.0041,2.2933e-07,2.7694e-07,2.8945e-07,2.9373e-07,2.9733e-07,1.4536e-06,1.9381e-06,4.8235e-06,4.8209e-06,4.8247e-06,4.8223e-06,1.2076,1.2622,1.0452,1.0606,1.0148,1.0272,1.0122,-8.3498e+06,540693.7038,-255606.5334,48177.6402,58140.3174,-3.8705,4.3914,10.1283,-5.9023,0.3165,-0.6332,-3.9224,-0.1401,25.6606,6.0260,5.7985,2.4085,0.0012,0.0795,100.5730,24.1476,0.0
4,24.4534,20.6992,19.0424,18.3242,17.9826,-3.3247e-05,18.1903,3.7543,24.4535,4.0196,1.6568,2.3750,-3.0722,1.8362,0.7182,1.0597,19.0424,-4.7290,-1.3915,0.8521,18.3242,-2.1097,-0.5388,-0.8803,-0.2076,0.1067,-18.1903,-17.8760,3.3376,4.9084,1.5709,2.2436,0.6727,0.9871,0.3144,1.1814,-7.3551e+05,1.0870,-6.2259e+05,0.8708,1.0973,1.0392,0.8011,0.9319,1.0190,-5.5115e+05,0.8968,0.9714,-5.4088e+05,0.9533,1.0060,-1.3986e-06,-1.6271e-06,-1.7626e-06,-1.8277e-06,-1.8599e-06,-2.5575e-06,-2.7706e-06,-2.7846e-05,-2.7838e-05,-2.7849e-05,-2.7843e-05,1.1633,1.2602,1.0833,1.1233,1.0370,1.0552,1.0176,1.2285e+06,-112285.4841,28776.9020,2431.2498,-1802.2420,3.5764,2.3899,5.9035,-2.4057,0.1657,1.1459,-0.3625,-0.5340,24.4534,6.4708,5.8955,2.4156,0.0005,0

In [11]:
trn['class'].value_counts()

2.0    156598
0.0    119532
1.0     42610
Name: class, dtype: int64

In [12]:
trn.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,class
count,318740.0000,318740.0000,318740.0000,318740.0000,318740.0000,3.1874e+05,318740.0000,318740.0000,318740.0000,318740.0000,318740.0000,318740.0000,318740.0000,318740.0000,318740.0000,318740.0000,318740.0000,318740.0000,318740.0000,318740.0000,318740.0000,318740.0000,318740.0000,318740.0000,318740.0000,318740.0000,318740.0000,318740.0000,318740.0000,318740.0000,318740.0000,318740.0000,318740.0000,318740.0000,318740.0000,318740.0000,3.1874e+05,318740.0000,3.1874e+05,318740.0000,318740.0000,318740.0000,318740.0000,318740.0000,318740.0000,3.1874e+05,318740.0000,318740.0000,3.1874e+05,318740.0000,318740.0000,3.1874e+05,3.1874e+05,3.1874e+05,3.1874e+05,3.1874e+05,3.1874e+05,3.1874e+05,3.1874e+05,3.1874e+05,3.1874e+05,3.1874e+05,318740.0000,318740.0000,318740.0000,318740.0000,318740.0000,318740.0000,318740.0000,3.1874e+05,3.1874e+05,3.1874e+05,318740.0000,318740.0000,318740.0000,318740.0000,318740.0000,318740.0000,318740.0000,318740.0000,318740.0000,318740.0000,318740.0000,318740.0000,318740.0000,318740.0000,3.1874e+05,318740.0000,318740.0000,318740.0000,318740.0000
mean,19.8463,18.4236,17.7278,17.2952,17.0517,5.9411e-02,17.2064,1.4226,19.7869,1.5949,0.6958,1.1284,-1.2013,0.8149,0.4326,0.6761,17.6684,-1.8971,-0.5235,0.5214,17.2358,-0.9561,-0.3135,-0.5570,-0.1548,0.0661,-17.1470,-16.9262,1.3736,2.0162,0.6426,1.0449,0.4023,0.6231,0.2208,1.0771,-1.1308e+04,1.0390,-1.0272e+04,0.9403,1.0464,1.0249,0.9058,0.9725,1.0146,-9.5646e+03,0.9497,0.9832,-9.4165e+03,0.9696,1.0040,3.0547e-03,3.2636e-03,3.3855e-03,3.4577e-03,3.5032e-03,3.1900e-02,3.2195e-02,5.1034e-02,5.1011e-02,5.1044e-02,5.1023e-02,1.0752,1.1145,1.0362,1.0606,1.0232,1.0371,1.0133,-1.0511e+05,3.5999e+02,3.1310e+02,-177.5487,157.8054,0.0372,-0.0317,-0.0017,-0.0044,0.0022,0.0077,0.0004,0.0059,19.8709,2.8584,2.7135,1.0998,6.7136e-03,0.0736,84.4657,18.4732,1.1163
std,1.9213,1.6394,1.4396,1.2904,1.2972,8.6936e-02,1.3012,0.7394,1.9282,0.8892,0.5254,0.8751,0.8238,0.6373,0.5111,0.7490,1.4434,1.0744,0.5944,0.5722,1.2926,0.8910,0.5512,0.7637,0.4387,0.1844,1.3024,1.3044,0.6977,1.0209,0.5072,0.8562,0.5041,0.7358,0.3946,0.0365,1.9728e+06,0.0273,1.7859e+06,0.0407,0.0370,0.0270,0.0484,0.0323,0.0211,1.6323e+06,0.0437,0.0293,1.6020e+06,0.0394,0.0115,4.2053e-03,4.4596e-03,4.6432e-03,4.7297e-03,4.7923e-03,5.2698e-02,5.3945e-02,7.3801e-02,7.3767e-02,7.3816e-02,7.3785e-02,0.0349,0.0549,0.0268,0.0471,0.0269,0.0413,0.0208,3.8885e+06,2.3250e+05,4.4247e+04,13193.4198,11662.9190,6.7234,3.0224,2.7889,2.6675,0.4842,2.0582,1.5638,1.3970,1.9350,1.5170,1.4562,0.5908,5.8399e-03,0.1927,7.0276,1.9033,0.9235
min,10.2007,10.8645,10.8451,10.7326,9.9532,-3.8073e+00,10.3629,-12.3568,10.2009,-12.2747,-15.6471,-13.0747,-13.0021,-14.3009,-13.5065,-14.4023,10.8453,-14.9659,-15.2121,-13.3791,10.6909,-14.9950,-15.6213,-13.6564,-13.9562,-7.6356,-31.3431,-28.5807,-12.3501,-11.9435,-11.7109,-13.1746,-13.5664,-14.6446,-13.6082,0.5913,-1.3211e+08,0.5328,-1.2412e+08,0.5289,0.5525,0.5273,0.4673,0.4966,0.4693,-1.2023e+08,0.4586,0.4849,-1.1982e+08,0.5138,0.6642,-1.8803e-01,-1.9445e-01,-2.1393e-01,-2.0578e-01,-2.1778e-01,-2.0037e+00,-2.0037e+00,-3.1868e+00,-3.1854e+00,-3.1873e+00,-3.1862e+00,0.5873,0.5950,0.5722,0.5369,0.5231,0.4819,0.4979,-2.8535e+08,-1.6531e+07,-3.0619e+06,-871806.1247,-742793.9774,-62.1158,-11.5240,-79.0295,-41.4254,-16.8422,-45.8010,-78.6759,-91.1503,11.0908,0.0844,0.0779,0.0365,1.9000e-05,-3.2741,47.5030,9.4039,0.0000
25%,18.7237,17.4738,16.8765,16.5231,16.2879,3.5660e-05,16.4524,1.0726,18.6646,1.1800,0.4278,0.6751,-1.4297,0.5057,0.2428,0.3561,16.8154,-2.2675,-0.7250,0.3059,16.4625,-1.1668,-0.3719,-0.6692,-0.2537,0.0208,-17.8455,-17.6003,1.0382,1.4188,0.3825,0.6024,0.2142,0.3081,0.0848,1.0595,9.3480e+01,1.0246,8.8376e+01,0.9261,1.0293,1.0141,0.8835,0.9

In [13]:
tst = pd.read_csv(tst_file, index_col = 0)
tst

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89
0,19.1431,18.0172,17.4733,17.1144,16.9263,1.2130e-01,17.0838,1.1259,19.0217,1.1881,0.5439,0.9028,-1.0578,0.5863,0.3588,0.5470,17.3520,-1.6017,-0.4817,0.3894,16.9931,-0.8405,-0.3164,-0.5046,-0.1575,0.0176,-16.9625,-16.7874,1.1200,1.6441,0.5241,0.8711,0.3470,0.5221,0.1751,1.0625,157.8113,1.0311,148.5300,0.9445,1.0336,1.0210,0.9160,0.9732,1.0111,141.0876,0.9532,0.9818,139.5368,0.9711,1.0010,6.3593e-03,6.7560e-03,6.9591e-03,7.1005e-03,7.1740e-03,1.2130e-01,1.2130e-01,1.0490e-01,1.0535e-01,1.0468e-01,1.0512e-01,1.0624,1.0943,1.0301,1.0510,1.0203,1.0309,1.0104,-1.2800e+05,1196.9230,235.6550,-139.1880,160.5277,-1.1646,-0.3764,-0.6646,-0.3363,0.2974,0.2247,0.1788,0.0961,19.1431,2.2168,2.1662,0.8708,9.8340e-03,0.0176,82.8923,17.9136
1,18.7409,17.1705,16.3691,15.9865,15.7307,-6.6907e-05,14.5160,1.5704,18.7410,4.4312,0.8014,1.1840,2.1001,2.7804,0.3826,0.6384,16.3692,1.2988,2.0593,1.8531,15.9865,1.6767,1.5963,1.3406,1.2147,1.0935,-14.5161,-14.6372,0.7606,0.6802,-0.0803,-0.2062,-0.1259,-0.2470,-0.1212,1.0915,-280103.3378,1.0490,-256631.4466,1.1394,1.1932,1.0239,1.0862,1.1439,1.0163,-238934.9505,1.1172,1.1109,-235112.5900,1.0932,1.0747,-4.4397e-06,-4.6756e-06,-4.6495e-06,-4.6092e-06,-4.5710e-06,-3.3454e-05,-3.3454e-05,-6.6400e-05,-6.6417e-05,-6.6390e-05,-6.6409e-05,1.0532,1.0473,0.9944,0.9858,0.9913,0.9831,0.9917,4.3461e+05,-18062.1118,1116.7790,-865.0933,751.4398,11.0658,2.4369,7.5851,14.9594,-0.4639,0.4538,-1.2207,-0.4063,18.7409,3.0102,0.7606,0.2986,5.9200e-04,1.4209,78.9593,14.0624
2,19.2980,17.8803,17.2221,16.8096,16.6352,1.1001e-01,16.7497,1.4177,19.1880,1.5254,0.6582,1.0708,-1.2200,0.7343,0.4126,0.5869,17.1121,-1.8782,-0.5505,0.4725,16.6996,-0.9631,-0.3364,-0.5108,-0.1144,0.0664,-16.6397,-16.4589,1.3277,1.9543,0.6266,1.0230,0.3963,0.5771,0.1808,1.0793,175.4203,1.0382,162.5333,0.9361,1.0428,1.0245,0.9017,0.9690,1.0105,152.7998,0.9458,0.9804,151.2152,0.9702,1.0040,5.7596e-03,6.1899e-03,6.4161e-03,6.5679e-03,6.6396e-03,1.1001e-01,1.1001e-01,7.9668e-02,7.9116e-02,7.9941e-02,7.9393e-02,1.0747,1.1140,1.0365,1.0611,1.0237,1.0348,1.0109,-1.2803e+05,1200.7489,234.4996,-139.4618,160.7233,-7.9098,-0.2344,0.1504,-0.0154,0.5051,0.2173,0.1883,0.1117,19.2980,2.6628,2.5315,1.0229,1.8988e-02,0.0664,80.9407,17.7099
3,18.0775,17.1569,16.9041,16.8118,16.7955,2.9697e-05,16.7700,0.9205,18.0774,1.0039,0.2528,0.3451,-0.8160,0.3100,0.0923,0.1086,16.9041,-1.0688,-0.1694,0.1342,16.8118,-0.2617,-0.0352,-0.0514,0.0256,0.0288,-16.7699,-16.7667,0.8994,1.1260,0.2266,0.3036,0.0770,0.0802,0.0032,1.0537,608734.6509,1.0150,577737.2243,0.9546,1.0184,1.0055,0.9405,0.9901,1.0010,566115.0006,0.9847,0.9979,565566.8063,0.9969,1.0017,1.6523e-06,1.7393e-06,1.7627e-06,1.7708e-06,1.7712e-06,2.9697e-05,2.9697e-05,2.6032e-05,2.6031e-05,2.6031e-05,2.6032e-05,1.0527,1.0668,1.0134,1.0181,1.0046,1.0048,1.0002,-1.4194e+06,-2626.0413,2741.7742,4617.5482,598.5527,-1.5521,-0.2674,-0.0394,-0.3483,-0.3142,0.2022,0.0634,0.1874,18.0775,1.2819,1.2062,0.5113,1.1600e-04,0.0288,80.0417,16.8320
4,17.8325,16.7646,16.2135,15.8722,15.6621,6.8471e-02,15.8315,1.0679,17.7640,1.1530,0.5511,0.8924,-0.9899,0.6083,0.3413,0.5514,16.1451,-1.5409,-0.4659,0.3821,15.8037,-0.8072,-0.2841,-0.4942,-0.1694,0.0187,-15.7630,-15.5749,1.0750,1.5982,0.5232,0.8480,0.3248,0.5129,0.1881,1.0637,260.4378,1.0340,244.8417,0.9442,1.0377,1.0215,0.9132,0.9721,1.0134,231.8084,0.9516,0.9824,228.7400,0.9694,1.0012,3.8566e-03,4.1051e-03,4.2381e-03,4.3250e-03,4.3770e-03,3.4236e-02,3.4236e-02,6.1629e-02,6.1816e-02,6.1533e-02,6.1723e-02,1.0645,1.0989,1.0324,1.0536,1.0205,1.0328,1.0120,-1.2821e+05,1201.4544,236.6646,-138.7002,160.2035,4.5367,0.1147,-1.5739,-0.0037,-0.2064,0.0311,0.3848,0.1094,17.8325,2.1704,2.1111,0.8466,6.8530e-03,0.0187,76.7896,16.6400
...,...,...,...,...,...,...,

In [14]:
tst.isna().sum()

0     0
1     0
2     0
3     0
4     0
     ..
85    0
86    0
87    0
88    0
89    0
Length: 90, dtype: int64

# 앙상블 전 모델 학습 

## 1. Stratified 10-Fold lgbm-gbdt

In [15]:
y = trn['class'].values
trn_df = trn.drop('class', axis = 1).values
tst_df = tst.values
trn_df.shape, y.shape, tst_df.shape

((318740, 90), (318740,), (80000, 90))

In [ ]:
n_class =3
n_fold = 10
p_val = np.zeros((trn_df.shape[0], n_class))
p_tst = np.zeros((tst_df.shape[0], n_class))

cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)
for i, (i_trn, i_val) in enumerate(cv.split(trn_df, y), 1):
   # print(i, i_trn, i_val)
    print(f'training model for CV #{i}')
    clf = lgb.LGBMClassifier(objective='multiclass',
                             boosting_type='gbdt',
                             n_estimators=15078,
                             num_leaves=97,
                             learning_rate=0.00814546,
                             num_iterations=573,
                             random_state=seed,
                             max_depth = 16,
                             #feature_fraction = 0.9,
                             #bagging_fraction = 0.7,
                             n_jobs=-1)
    clf.fit(trn_df[i_trn], y[i_trn],
            eval_set=[(trn_df[i_val], y[i_val])],
            eval_metric='multiclass',
            early_stopping_rounds=100)
    
    p_val[i_val, :] = clf.predict_proba(trn_df[i_val])
    p_tst += clf.predict_proba(tst_df) / n_fold
    
    del clf, i_trn, i_val
    

print(f'{accuracy_score(y, np.argmax(p_val, axis=1)) * 100:.4f}%')


training model for CV #1
[1]	valid_0's multi_logloss: 0.973439
Training until validation scores don't improve for 100 rounds
[2]	valid_0's multi_logloss: 0.961169
[3]	valid_0's multi_logloss: 0.949176
[4]	valid_0's multi_logloss: 0.937462
[5]	valid_0's multi_logloss: 0.92599
[6]	valid_0's multi_logloss: 0.914773
[7]	valid_0's multi_logloss: 0.903782
[8]	valid_0's multi_logloss: 0.893024
[9]	valid_0's multi_logloss: 0.882497
[10]	valid_0's multi_logloss: 0.872176
[11]	valid_0's multi_logloss: 0.862071
[12]	valid_0's multi_logloss: 0.852148
[13]	valid_0's multi_logloss: 0.842411
[14]	valid_0's multi_logloss: 0.832877
[15]	valid_0's multi_logloss: 0.823537
[16]	valid_0's multi_logloss: 0.814356
[17]	valid_0's multi_logloss: 0.805332
[18]	valid_0's multi_logloss: 0.796489
[19]	valid_0's multi_logloss: 0.787812
[20]	valid_0's multi_logloss: 0.779285
[21]	valid_0's multi_logloss: 0.770908
[22]	valid_0's multi_logloss: 0.762679
[23]	valid_0's multi_logloss: 0.754602
[24]	valid_0's multi_loglo

[207]	valid_0's multi_logloss: 0.234432
[208]	valid_0's multi_logloss: 0.233667
[209]	valid_0's multi_logloss: 0.232911
[210]	valid_0's multi_logloss: 0.232165
[211]	valid_0's multi_logloss: 0.231425
[212]	valid_0's multi_logloss: 0.230693
[213]	valid_0's multi_logloss: 0.229969
[214]	valid_0's multi_logloss: 0.229255
[215]	valid_0's multi_logloss: 0.228547
[216]	valid_0's multi_logloss: 0.22785
[217]	valid_0's multi_logloss: 0.227155
[218]	valid_0's multi_logloss: 0.226468
[219]	valid_0's multi_logloss: 0.225788
[220]	valid_0's multi_logloss: 0.225119
[221]	valid_0's multi_logloss: 0.224454
[222]	valid_0's multi_logloss: 0.223798
[223]	valid_0's multi_logloss: 0.22315
[224]	valid_0's multi_logloss: 0.222509
[225]	valid_0's multi_logloss: 0.221874
[226]	valid_0's multi_logloss: 0.221237
[227]	valid_0's multi_logloss: 0.220618
[228]	valid_0's multi_logloss: 0.220004
[229]	valid_0's multi_logloss: 0.21939
[230]	valid_0's multi_logloss: 0.218786
[231]	valid_0's multi_logloss: 0.218189
[23

[413]	valid_0's multi_logloss: 0.16792
[414]	valid_0's multi_logloss: 0.167819
[415]	valid_0's multi_logloss: 0.167715
[416]	valid_0's multi_logloss: 0.167613
[417]	valid_0's multi_logloss: 0.167504
[418]	valid_0's multi_logloss: 0.167406
[419]	valid_0's multi_logloss: 0.167309
[420]	valid_0's multi_logloss: 0.16721
[421]	valid_0's multi_logloss: 0.16711
[422]	valid_0's multi_logloss: 0.167006
[423]	valid_0's multi_logloss: 0.166913
[424]	valid_0's multi_logloss: 0.16682
[425]	valid_0's multi_logloss: 0.166728
[426]	valid_0's multi_logloss: 0.166641
[427]	valid_0's multi_logloss: 0.166545
[428]	valid_0's multi_logloss: 0.166453
[429]	valid_0's multi_logloss: 0.166357
[430]	valid_0's multi_logloss: 0.166271
[431]	valid_0's multi_logloss: 0.166184
[432]	valid_0's multi_logloss: 0.166094
[433]	valid_0's multi_logloss: 0.166004
[434]	valid_0's multi_logloss: 0.165912
[435]	valid_0's multi_logloss: 0.165827
[436]	valid_0's multi_logloss: 0.16574
[437]	valid_0's multi_logloss: 0.165648
[438]

[43]	valid_0's multi_logloss: 0.61784
[44]	valid_0's multi_logloss: 0.612108
[45]	valid_0's multi_logloss: 0.606459
[46]	valid_0's multi_logloss: 0.600901
[47]	valid_0's multi_logloss: 0.595427
[48]	valid_0's multi_logloss: 0.59003
[49]	valid_0's multi_logloss: 0.584716
[50]	valid_0's multi_logloss: 0.579481
[51]	valid_0's multi_logloss: 0.574329
[52]	valid_0's multi_logloss: 0.569243
[53]	valid_0's multi_logloss: 0.564232
[54]	valid_0's multi_logloss: 0.559299
[55]	valid_0's multi_logloss: 0.554425
[56]	valid_0's multi_logloss: 0.549627
[57]	valid_0's multi_logloss: 0.544898
[58]	valid_0's multi_logloss: 0.540247
[59]	valid_0's multi_logloss: 0.53565
[60]	valid_0's multi_logloss: 0.53112
[61]	valid_0's multi_logloss: 0.526661
[62]	valid_0's multi_logloss: 0.522267
[63]	valid_0's multi_logloss: 0.517935
[64]	valid_0's multi_logloss: 0.513668
[65]	valid_0's multi_logloss: 0.509458
[66]	valid_0's multi_logloss: 0.5053
[67]	valid_0's multi_logloss: 0.501205
[68]	valid_0's multi_logloss: 0

[250]	valid_0's multi_logloss: 0.206749
[251]	valid_0's multi_logloss: 0.20626
[252]	valid_0's multi_logloss: 0.205777
[253]	valid_0's multi_logloss: 0.205302
[254]	valid_0's multi_logloss: 0.204829
[255]	valid_0's multi_logloss: 0.204359
[256]	valid_0's multi_logloss: 0.203899
[257]	valid_0's multi_logloss: 0.203442
[258]	valid_0's multi_logloss: 0.202991
[259]	valid_0's multi_logloss: 0.202541
[260]	valid_0's multi_logloss: 0.2021
[261]	valid_0's multi_logloss: 0.201665
[262]	valid_0's multi_logloss: 0.201232
[263]	valid_0's multi_logloss: 0.200807
[264]	valid_0's multi_logloss: 0.200384
[265]	valid_0's multi_logloss: 0.199956
[266]	valid_0's multi_logloss: 0.19954
[267]	valid_0's multi_logloss: 0.199124
[268]	valid_0's multi_logloss: 0.198714
[269]	valid_0's multi_logloss: 0.198304
[270]	valid_0's multi_logloss: 0.1979
[271]	valid_0's multi_logloss: 0.1975
[272]	valid_0's multi_logloss: 0.197108
[273]	valid_0's multi_logloss: 0.196719
[274]	valid_0's multi_logloss: 0.196333
[275]	va

...

[310]	valid_0's multi_logloss: 0.187924
[311]	valid_0's multi_logloss: 0.187656
[312]	valid_0's multi_logloss: 0.187395
[313]	valid_0's multi_logloss: 0.187131
[314]	valid_0's multi_logloss: 0.186876
[315]	valid_0's multi_logloss: 0.186621
[316]	valid_0's multi_logloss: 0.18637
[317]	valid_0's multi_logloss: 0.186116
[318]	valid_0's multi_logloss: 0.185861
[319]	valid_0's multi_logloss: 0.185613
[320]	valid_0's multi_logloss: 0.185366
[321]	valid_0's multi_logloss: 0.185121
[322]	valid_0's multi_logloss: 0.184877
[323]	valid_0's multi_logloss: 0.184637
[324]	valid_0's multi_logloss: 0.184398
[325]	valid_0's multi_logloss: 0.184163
[326]	valid_0's multi_logloss: 0.183932
[327]	valid_0's multi_logloss: 0.183701
[328]	valid_0's multi_logloss: 0.183476
[329]	valid_0's multi_logloss: 0.183256
[330]	valid_0's multi_logloss: 0.183035
[331]	valid_0's multi_logloss: 0.182821
[332]	valid_0's multi_logloss: 0.182604
[333]	valid_0's multi_logloss: 0.182384
[334]	valid_0's multi_logloss: 0.182166
[

[516]	valid_0's multi_logloss: 0.162552
[517]	valid_0's multi_logloss: 0.162505
[518]	valid_0's multi_logloss: 0.162458
[519]	valid_0's multi_logloss: 0.16241
[520]	valid_0's multi_logloss: 0.162369
[521]	valid_0's multi_logloss: 0.162316
[522]	valid_0's multi_logloss: 0.162267
[523]	valid_0's multi_logloss: 0.16222
[524]	valid_0's multi_logloss: 0.162178
[525]	valid_0's multi_logloss: 0.16214
[526]	valid_0's multi_logloss: 0.1621
[527]	valid_0's multi_logloss: 0.162057
[528]	valid_0's multi_logloss: 0.162018
[529]	valid_0's multi_logloss: 0.161982
[530]	valid_0's multi_logloss: 0.161942
[531]	valid_0's multi_logloss: 0.161901
[532]	valid_0's multi_logloss: 0.161863
[533]	valid_0's multi_logloss: 0.161819
[534]	valid_0's multi_logloss: 0.16178
[535]	valid_0's multi_logloss: 0.161741
[536]	valid_0's multi_logloss: 0.161701
[537]	valid_0's multi_logloss: 0.161661
[538]	valid_0's multi_logloss: 0.161618
[539]	valid_0's multi_logloss: 0.161583
[540]	valid_0's multi_logloss: 0.161539
[541]	

[147]	valid_0's multi_logloss: 0.297152
[148]	valid_0's multi_logloss: 0.295664
[149]	valid_0's multi_logloss: 0.294198
[150]	valid_0's multi_logloss: 0.292746
[151]	valid_0's multi_logloss: 0.291306
[152]	valid_0's multi_logloss: 0.289882
[153]	valid_0's multi_logloss: 0.288475
[154]	valid_0's multi_logloss: 0.287083
[155]	valid_0's multi_logloss: 0.285712
[156]	valid_0's multi_logloss: 0.284357
[157]	valid_0's multi_logloss: 0.283015
[158]	valid_0's multi_logloss: 0.281686
[159]	valid_0's multi_logloss: 0.280374
[160]	valid_0's multi_logloss: 0.279074
[161]	valid_0's multi_logloss: 0.277791
[162]	valid_0's multi_logloss: 0.276517
[163]	valid_0's multi_logloss: 0.275264
[164]	valid_0's multi_logloss: 0.274021
[165]	valid_0's multi_logloss: 0.272795
[166]	valid_0's multi_logloss: 0.27158
[167]	valid_0's multi_logloss: 0.270381
[168]	valid_0's multi_logloss: 0.269196
[169]	valid_0's multi_logloss: 0.26802
[170]	valid_0's multi_logloss: 0.266861
[171]	valid_0's multi_logloss: 0.265716
[1

[353]	valid_0's multi_logloss: 0.17363
[354]	valid_0's multi_logloss: 0.173453
[355]	valid_0's multi_logloss: 0.173275
[356]	valid_0's multi_logloss: 0.173102
[357]	valid_0's multi_logloss: 0.172929
[358]	valid_0's multi_logloss: 0.172762
[359]	valid_0's multi_logloss: 0.172595
[360]	valid_0's multi_logloss: 0.17243
[361]	valid_0's multi_logloss: 0.172262
[362]	valid_0's multi_logloss: 0.172097
[363]	valid_0's multi_logloss: 0.171931
[364]	valid_0's multi_logloss: 0.17177
[365]	valid_0's multi_logloss: 0.171608
[366]	valid_0's multi_logloss: 0.171448
[367]	valid_0's multi_logloss: 0.171287
[368]	valid_0's multi_logloss: 0.171127
[369]	valid_0's multi_logloss: 0.170971
[370]	valid_0's multi_logloss: 0.170817
[371]	valid_0's multi_logloss: 0.170657
[372]	valid_0's multi_logloss: 0.170501
[373]	valid_0's multi_logloss: 0.170345
[374]	valid_0's multi_logloss: 0.170192
[375]	valid_0's multi_logloss: 0.170043
[376]	valid_0's multi_logloss: 0.169895
[377]	valid_0's multi_logloss: 0.16974
[378

[559]	valid_0's multi_logloss: 0.156008
[560]	valid_0's multi_logloss: 0.155973
[561]	valid_0's multi_logloss: 0.155941
[562]	valid_0's multi_logloss: 0.15591
[563]	valid_0's multi_logloss: 0.155874
[564]	valid_0's multi_logloss: 0.155842
[565]	valid_0's multi_logloss: 0.155811
[566]	valid_0's multi_logloss: 0.155778
[567]	valid_0's multi_logloss: 0.155743
[568]	valid_0's multi_logloss: 0.15571
[569]	valid_0's multi_logloss: 0.155681
[570]	valid_0's multi_logloss: 0.15565
[571]	valid_0's multi_logloss: 0.155619
[572]	valid_0's multi_logloss: 0.155587
[573]	valid_0's multi_logloss: 0.155556
Did not meet early stopping. Best iteration is:
[573]	valid_0's multi_logloss: 0.155556
training model for CV #10
[1]	valid_0's multi_logloss: 0.973434
Training until validation scores don't improve for 100 rounds
[2]	valid_0's multi_logloss: 0.961191
[3]	valid_0's multi_logloss: 0.949222
[4]	valid_0's multi_logloss: 0.937528
[5]	valid_0's multi_logloss: 0.926089
[6]	valid_0's multi_logloss: 0.914908

In [ ]:
np.savetxt(st_data_dir/'Stratified_10-Fold_lgbm-gbdt_val.csv', p_val, fmt='%.6f', delimiter=',')
np.savetxt(st_data_dir/'Stratified_10-Fold_lgbm-gbdt_tst.csv', p_tst, fmt='%.6f', delimiter=',')

# files.download('Stratified_10-Fold_lgbm-gbdt_val.csv')
# files.download('Stratified_10-Fold_lgbm-gbdt_tst.csv')

In [ ]:
sub = pd.read_csv(sample_file, index_col=0)
sub

In [ ]:
sub[target_col] = np.argmax(p_tst, axis=1)

In [ ]:
sub[target_col].value_counts()

In [ ]:
sub.to_csv(submission_dir/'Stratified_10-Fold_lgbm-gbdt_submission.csv')
# files.download('Stratified_10-Fold_lgbm-gbdt_submission.csv')

## 2. Stratified 10-Fold lgbm-dart

In [ ]:
y = trn['class'].values
trn_df = trn.drop('class', axis = 1).values
tst_df = tst.values
trn_df.shape, y.shape, tst_df.shape

In [ ]:
n_class =3
n_fold = 10
p_val = np.zeros((trn_df.shape[0], n_class))
p_tst = np.zeros((tst_df.shape[0], n_class))

cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)
for i, (i_trn, i_val) in enumerate(cv.split(trn_df, y), 1):
   # print(i, i_trn, i_val)
    print(f'training model for CV #{i}')
    clf = lgb.LGBMClassifier(objective='multiclass',
                             boosting_type='dart',
                             n_estimators=30000,
                             num_leaves=256,
                             learning_rate=0.01,
                             min_child_samples=10,
                             subsample=.5,
                             subsample_freq=1,
                             colsample_bytree=.8,
                             random_state=seed,
                             max_depth = 27,
                             #feature_fraction = 0.9,
                             #bagging_fraction = 0.7,
                             n_jobs=-1)
    clf.fit(trn_df[i_trn], y[i_trn],
            eval_set=[(trn_df[i_val], y[i_val])],
            eval_metric='multiclass',
            early_stopping_rounds=50)
    
    p_val[i_val, :] = clf.predict_proba(trn_df[i_val])
    p_tst += clf.predict_proba(tst_df) / n_fold
    
    del clf, i_trn, i_val
    

print(f'{accuracy_score(y, np.argmax(p_val, axis=1)) * 100:.4f}%')

In [ ]:
np.savetxt(st_data_dir/'Stratified_10-Fold_lgbm-dart_val.csv', p_val, fmt='%.6f', delimiter=',')
np.savetxt(st_data_dir/'Stratified_10-Fold_lgbm-dart_tst.csv', p_tst, fmt='%.6f', delimiter=',')

# files.download('Stratified_10-Fold_lgbm-dart_val.csv')
# files.download('Stratified_10-Fold_lgbm-dart_tst.csv')

In [ ]:
sub = pd.read_csv(sample_file, index_col=0)
sub

In [ ]:
sub[target_col] = np.argmax(p_tst, axis=1)

In [ ]:
sub[target_col].value_counts()

In [ ]:
sub.to_csv(submission_dir/'Stratified_10-Fold_lgbm-dart_submission.csv')
# files.download('Stratified_10-Fold_lgbm-dart_submission.csv')

# 3. Stratified 10-fold RandomForest

In [ ]:
y = trn['class'].values
trn_df = trn.drop('class', axis = 1).values
tst_df = tst.values
trn_df.shape, y.shape, tst_df.shape

In [ ]:
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

p_val = np.zeros((trn.shape[0], n_class))
p_tst = np.zeros((tst.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(trn_df, y), 1):
    print(f'training model for CV #{i}')
    clf = RandomForestClassifier(n_estimators=1000,
                                 min_samples_leaf=10,
                                 max_features='auto',
                                 max_samples=.5,
                                 random_state=seed,
                                 n_jobs=-1)
    clf.fit(trn_df[i_trn], y[i_trn])
    p_val[i_val, :] = clf.predict_proba(trn_df[i_val])
    p_tst += clf.predict_proba(tst_df) / n_fold

    del clf, i_trn, i_val
print(f'{accuracy_score(y, np.argmax(p_val, axis=1)) * 100:.4f}%')

In [ ]:
np.savetxt(st_data_dir/'Stratified_10-fold_RandomForest_val.csv', p_val, fmt='%.6f', delimiter=',')
np.savetxt(st_data_dir/'Stratified_10-fold_RandomForest_tst.csv', p_tst, fmt='%.6f', delimiter=',')
# files.download('Stratified_10-fold_RandomForest_val.csv')
# files.download('Stratified_10-fold_RandomForest_tst.csv')

In [ ]:
sub = pd.read_csv(sample_file, index_col=0)
sub

In [ ]:
sub[target_col] = np.argmax(p_tst, axis=1)

In [ ]:
sub[target_col].value_counts()

In [ ]:
sub.to_csv(submission_dir/'Stratified_10-fold_RandomForest_submission.csv')
# files.download('Stratified_10-fold_RandomForest_submission.csv')

# 4. Stratified 10-fold DecisionTree

In [ ]:
y = trn['class'].values
trn_df = trn.drop('class', axis = 1).values
tst_df = tst.values
trn_df.shape, y.shape, tst_df.shape

In [ ]:
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

p_val = np.zeros((trn.shape[0], n_class))
p_tst = np.zeros((tst.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(trn_df, y), 1):
    print(f'training model for CV #{i}')
    clf = DecisionTreeClassifier(max_depth=5, 
                                 min_samples_leaf=10, 
                                 random_state=seed)
    clf.fit(trn_df[i_trn], y[i_trn])
    p_val[i_val, :] = clf.predict_proba(trn_df[i_val])
    p_tst += clf.predict_proba(tst_df) / n_fold

    del clf, i_trn, i_val
print(f'{accuracy_score(y, np.argmax(p_val, axis=1)) * 100:.4f}%')

In [ ]:
np.savetxt(st_data_dir/'Stratified_10-fold_DecisionTree_val.csv', p_val, fmt='%.6f', delimiter=',')
np.savetxt(st_data_dir/'Stratified_10-fold_DecisionTree_tst.csv', p_tst, fmt='%.6f', delimiter=',')
# files.download('Stratified_10-fold_DecisionTree_val.csv')
# files.download('Stratified_10-fold_DecisionTree_tst.csv')

In [ ]:
sub = pd.read_csv(sample_file, index_col=0)
sub

In [ ]:
sub[target_col] = np.argmax(p_tst, axis=1)

In [ ]:
sub[target_col].value_counts()

In [ ]:
sub.to_csv(submission_dir/'Stratified_10-fold_DecisionTree_submission.csv')
# files.download('Stratified_10-fold_DecisionTree_submission.csv')

# 5. Stratified 10-Fold Logistic Regresstion

In [ ]:
y = trn['class'].values
trn_df = trn.drop('class', axis = 1).values
tst_df = tst.values
trn_df.shape, y.shape, tst_df.shape

In [ ]:
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

p_val = np.zeros((trn.shape[0], n_class))
p_tst = np.zeros((tst.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(trn_df, y), 1):
    print(f'training model for CV #{i}')
    clf = LogisticRegression(multi_class='multinomial')
    clf.fit(trn_df[i_trn], y[i_trn])
    p_val[i_val, :] = clf.predict_proba(trn_df[i_val])
    p_tst += clf.predict_proba(tst_df) / n_fold

    del clf, i_trn, i_val
print(f'{accuracy_score(y, np.argmax(p_val, axis=1)) * 100:.4f}%')

In [ ]:
np.savetxt(st_data_dir/'Stratified_10-fold_LogsticRegression_val.csv', p_val, fmt='%.6f', delimiter=',')
np.savetxt(st_data_dir/'Stratified_10-fold_LogsticRegression_tst.csv', p_tst, fmt='%.6f', delimiter=',')
# files.download('Stratified_10-fold_LogsticRegression_val.csv')
# files.download('Stratified_10-fold_LogsticRegression_tst.csv')

In [ ]:
sub = pd.read_csv(sample_file, index_col=0)
sub

In [ ]:
sub[target_col] = np.argmax(p_tst, axis=1)

In [ ]:
sub[target_col].value_counts()

In [ ]:
sub.to_csv(submission_dir/'Stratified_10-fold_LogsticRegression_submission.csv')
# files.download('Stratified_10-fold_LogsticRegression_submission.csv')

# 6. Stratified_10-fold_XGBoost

In [ ]:
y = trn['class'].values
trn_df = trn.drop('class', axis = 1).values
tst_df = tst.values
trn_df.shape, y.shape, tst_df.shape

In [ ]:
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

p_val = np.zeros((trn.shape[0], n_class))
p_tst = np.zeros((tst.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(trn_df, y), 1):
    print(f'training model for CV #{i}')
    clf = xgb.XGBClassifier(max_depth = 27, 
                            learning_rate = 0.01, 
                            n_estimators = 1050)
    clf.fit(trn_df[i_trn], y[i_trn])
    p_val[i_val, :] = clf.predict_proba(trn_df[i_val])
    p_tst += clf.predict_proba(tst_df) / n_fold

    del clf, i_trn, i_val
print(f'{accuracy_score(y, np.argmax(p_val, axis=1)) * 100:.4f}%')

In [ ]:
np.savetxt(st_data_dir/'Stratified_10-fold_XGBoost_val.csv', p_val, fmt='%.6f', delimiter=',')
np.savetxt(st_data_dir/'Stratified_10-fold_XGBoost_tst.csv', p_tst, fmt='%.6f', delimiter=',')
# files.download('Stratified_10-fold_XGBoost_val.csv')
# files.download('Stratified_10-fold_XGBoost_tst.csv')

In [ ]:
sub = pd.read_csv(sample_file, index_col=0)
sub

In [ ]:
sub[target_col] = np.argmax(p_tst, axis=1)

In [ ]:
sub[target_col].value_counts()

In [ ]:
sub.to_csv(submission_dir/'Stratified_10-fold_XGBoost_submission.csv')
# files.download('Stratified_10-fold_XGBoost_submission.csv')

# 7. Stratified 10-fold NN

In [55]:
temp = trn.drop(target_col, axis = 1)
trn_nn = Normalization_df(Standardzation_df(temp))
trn_nn[target_col] = trn[target_col]
tst_nn = Normalization_df(Standardzation_df(tst))


In [76]:
y = LabelEncoder().fit_transform(trn_nn['class'])
y_label = y.copy()
y = to_categorical(y)

trn_df = trn_nn.copy().drop('class', axis = 1).values
tst_df = tst_nn.values
trn_df.shape, y.shape, tst_df.shape

((318740, 90), (318740, 3), (80000, 90))

In [83]:
#model layers

model = Sequential()

model.add(Dense(258, input_dim=90, activation='relu'))

model.add(Dense(258, activation='relu'))

model.add(Dense(3, activation='sigmoid'))

model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy',])

es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='min',
    baseline=None, restore_best_weights=True
)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 258)               23478     
_________________________________________________________________
dense_13 (Dense)             (None, 258)               66822     
_________________________________________________________________
dense_14 (Dense)             (None, 3)                 777       
Total params: 91,077
Trainable params: 91,077
Non-trainable params: 0
_________________________________________________________________


In [ ]:
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed, )

p_val = np.zeros((trn.shape[0], n_class))
p_tst = np.zeros((tst.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(trn_df, y.argmax(1)), 1):
    print(f'training model for CV #{i}')
    clf = model
    clf.fit(trn_df[i_trn], y[i_trn], epochs=300, batch_size=64, callbacks=[es])
    p_val[i_val, :] = clf.predict(trn_df[i_val])
    p_tst += clf.predict(tst_df) / n_fold

    del clf, i_trn, i_val
print(f'{accuracy_score(y, np.argmax(p_val, axis=1)) * 100:.4f}%')

training model for CV #1
Epoch 1/300
4483/4483 [==============================] - 10s 2ms/step - loss: 0.4669 - accuracy: 0.8224
Epoch 2/300
1570/4483 [=========>....................] - ETA: 6s - loss: 0.3571 - accuracy: 0.8680

In [ ]:
np.savetxt(st_data_dir/'Stratified_10-fold_NN_val.csv', p_val, fmt='%.6f', delimiter=',')
np.savetxt(st_data_dir/'Stratified_10-fold_NN_tst.csv', p_tst, fmt='%.6f', delimiter=',')
# files.download('Stratified_10-fold_NN_val.csv')
# files.download('Stratified_10-fold_NN_tst.csv')

In [ ]:
sub = pd.read_csv(sample_file, index_col=0)
sub

In [ ]:
sub[target_col] = np.argmax(p_tst, axis=1)

In [ ]:
sub[target_col].value_counts()

In [ ]:
sub.to_csv(submission_dir/'Stratified_10-fold_NN_submission.csv')
# files.download('Stratified_10-fold_NN_submission.csv')

# 8. 3등_model

In [ ]:
y = trn['class'].values
trn_df = trn.drop('class', axis = 1).values
tst_df = tst.values

X_trn, X_val, y_trn, y_val = train_test_split(trn_df, y, test_size=.01, random_state=seed)
print(X_trn.shape, X_val.shape, y_trn.shape, y_val.shape)

In [ ]:
clf = lgb.LGBMClassifier(objective='multiclass',
                         n_estimators=12000,
                         num_leaves=512,
                         learning_rate=0.001,
                         min_child_samples=10,
                         subsample=.5,
                         subsample_freq=1,
                         colsample_bytree=.8,
                         random_state=seed,
                         max_depth = 27,
                         n_jobs=-1)
                        #boosting_type = 'goss')
clf.fit(X_trn, y_trn,
        eval_set=[(X_val, y_val)],
        eval_metric='multiclass',
        early_stopping_rounds=100)

p_val = clf.predict_proba(tst_df)
p_tst = clf.predict_proba(tst_df)

print(f'{accuracy_score(y_val, p_val) * 100:.4f}%')

In [ ]:
np.savetxt(st_data_dir/'3rd_val.csv', p_val, fmt='%.6f', delimiter=',')
np.savetxt(st_data_dir/'3rd_tst.csv', p_tst, fmt='%.6f', delimiter=',')

In [ ]:
sub = pd.read_csv(sample_file, index_col=0)
sub

In [ ]:
sub[target_col] = np.argmax(p_tst, axis=1)

In [ ]:
sub[target_col].value_counts()

In [ ]:
sub.to_csv(submission_dir/'3rd_submission.csv')

# 앙상블 Stacking

# Stacking Feature 생성

In [ ]:
model_names = [
               'Stratified_10-Fold_lgbm-gbdt',
               'Stratified_10-Fold_lgbm-dart',
               'Stratified_10-fold_RandomForest',
               'Stratified_10-fold_DecisionTree',
               'Stratified_10-fold_LogsticRegression',
               'Stratified_10-fold_XGBoost',
               'Stratified_10-fold_NN',
               '3rd'
               ]
trn_st = []
tst_st = []
feature_names = [] 
for model in model_names:
    trn_st.append(np.loadtxt(f'{model}_val.csv',delimiter=','))
    tst_st.apeend(np.loadtxt(f'{model}_tst.csv'),delimiter=','))
    feature_names += [f'{model}_class0',f'{model}_class1',f'{model}_class2']

trn_st = np.hstack(trn_st) # training 용 데이터
tst_st = np.hstack(tst_st) # 실제 test 용 데이터

feature_names


In [ ]:
# training 예측 값 불러오기
y = pd.read_csv(trn_file, index_col=0, usecols=['id', target_col]).values.flatten()
y.shape

### trn_st(피쳐 수=90), tst_st(피쳐 수=90), y 를 쓰면 됨 

## 1. Stacking 10-fold lgbm-gbdt

In [ ]:
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

p_val = np.zeros((trn.shape[0], n_class))
p_tst = np.zeros((tst.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(trn_st, y), 1):
    print(f'training model for CV #{i}')
    clf = lgb.LGBMClassifier(objective='multiclass',
                             boosting_type='gbdt',
                             n_estimators=20000,
                             num_leaves=512,
                             learning_rate=0.001,
                             min_child_samples=10,
                             subsample=.5,
                             subsample_freq=1,
                             colsample_bytree=.8,
                             random_state=seed,
                             max_depth = 27,
                             #feature_fraction = 0.9,
                             #bagging_fraction = 0.7,
                             n_jobs=-1)
    clf.fit(trn[i_trn], y[i_trn],
            eval_set=[(trn_st[i_val], y[i_val])],
            eval_metric='multiclass',
            early_stopping_rounds=10)
    
    p_val[i_val, :] = clf.predict_proba(trn_st[i_val])
    p_tst += clf.predict_proba(tst_st) / n_fold

In [ ]:
np.savetxt(st_data_dir/'Stacking_Stratified_10-Fold_lgbm-gbdt_val.csv', p_val, fmt='%.6f', delimiter=',')
np.savetxt(st_data_dir/'Stacking_Stratified_10-Fold_lgbm-gbdt_tst.csv', p_tst, fmt='%.6f', delimiter=',')
# files.download('Stacking_Stratified_10-Fold_lgbm-gbdt_val.csv')
# files.download('Stacking_Stratified_10-Fold_lgbm-gbdt_tst.csv')

In [ ]:
clf.coef_.shape

In [ ]:
imp = pd.DataFrame({'feature': feature_names, 'importance': clf.feature_importances_})
imp = imp.sort_values('importance').set_index('feature')
imp.plot(kind='barh')

In [ ]:
sub = pd.read_csv(sample_file, index_col=0)
sub

In [ ]:
sub[target_col] = np.argmax(p_tst, axis=1)

In [ ]:
sub[target_col].value_counts()

In [ ]:
sub.to_csv(submission_dir/'Stacking_Stratified_10-Fold_lgbm-gbdt_sub.csv')
# files.download('Stacking_Stratified_10-Fold_lgbm-gbdt_sub.csv')